In [1]:
import numpy as np
from darli.model import Model, Functional
from darli.backend import CasadiBackend
import casadi as cs


In [2]:
%%capture
!pip3 install robot_descriptions
from robot_descriptions import iiwa_description, z1_description

In [3]:
model = Functional(CasadiBackend(z1_description.URDF_PATH))


In [4]:
model.add_body({'ee': "link06"})

In [5]:
opti_problem = cs.Opti()
q = opti_problem.variable(model.nq)

In [6]:
pos = model.body('ee').position(q)
gravity = model.inverse_dynamics(q, 0, 0)

In [7]:
# TODO: move to parameters
desired_pos = opti_problem.parameter(3)


In [8]:
cost = cs.sumsqr(pos - desired_pos) + 0.00*cs.sumsqr(gravity)
opti_problem.minimize(cost)
opti_problem.subject_to(opti_problem.bounded(model.q_min, q, model.q_max))
opti_problem.solver('ipopt')

In [9]:
opti_problem.set_value(desired_pos,[0.2, 0.2, 0.36])

In [10]:
opti_problem.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       15

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

OptiSol(Opti {
  instance #0
  #variables: 1 (nx = 6)
  #parameters: 1 (np = 3)
  #constraints: 1 (ng = 6)
  CasADi solver allocated.
  CasADi solver was called: Solve_Succeeded
})

In [11]:
q_opt = opti_problem.value(q)
q_opt

array([ 0.80816954,  1.44786167, -1.05499299,  0.13724142, -0.1312743 ,
        0.        ])

In [12]:
np.array(model.body('ee').position(q_opt))

array([[0.2 ],
       [0.2 ],
       [0.36]])

In [13]:
# opti_problem.solver('sqpmethod')
opti_problem.set_initial(q,q_opt)
opti_problem.set_value(desired_pos,[0.19, 0.21, 0.37])
opti_problem.solve()

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       15

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        6
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.9999999e-04 0.00e+00 4.00e-03  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

OptiSol(Opti {
  instance #0
  #variables: 1 (nx = 6)
  #parameters: 1 (np = 3)
  #constraints: 1 (ng = 6)
  CasADi solver allocated.
  CasADi solver was called: Solve_Succeeded
})